<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/prueba_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prueba de modelos

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
import torch
torch.cuda.is_available()

False

In [4]:
from transformers import pipeline

In [2]:
data = ["hay que matarlos a todos", "Vamos a la plaza a marchar","Los chinos son todos sucios","Culpa de los bolitas"]

### Modelo 1 BETO binario

Parametros:

output_dir=repo_name,
eval_strategy ="steps",
adam_beta1=0.9,
adam_beta2=0.98,
eval_steps=10000,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
weight_decay=0.01,
push_to_hub=True,

In [5]:
specific_model = pipeline(model="natdebandi/beto_hate_speech_ar_bin")
specific_model(data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/735k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.9985326528549194},
 {'label': 'LABEL_0', 'score': 0.9996645450592041},
 {'label': 'LABEL_1', 'score': 0.9978051781654358},
 {'label': 'LABEL_1', 'score': 0.999784529209137}]

### Modelo 2 BETO multietiqueta




In [38]:

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "natdebandi/beto_hate_speech_ar_multiclasif"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = "Hay que matarlos a todos, esos villeros son sucios"

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)

outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.7811,  4.9270, -3.7084, -3.5966,  1.3526,  5.2667, -1.4951, -3.5367,
          0.4954, -0.5906]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [39]:
model.config.id2label

{0: 'HATEFUL',
 1: 'CALLS',
 2: 'WOMEN',
 3: 'LGBTI',
 4: 'RACISM',
 5: 'CLASS',
 6: 'POLITICS',
 7: 'DISABLED',
 8: 'APPEARANCE',
 9: 'CRIMINAL'}

In [40]:
logits = outputs.logits
logits.shape

torch.Size([1, 10])

In [41]:
import numpy as np

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1


In [42]:
predictions

array([1., 1., 0., 0., 1., 1., 0., 0., 1., 0.])

In [43]:
# turn predicted id's into actual label names
predicted_labels = [model.config.id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

##NO PUDE METER MAS DE UN TEXTO

['HATEFUL', 'CALLS', 'RACISM', 'CLASS', 'APPEARANCE']
